In [7]:
from huggingface_hub import hf_hub_download
import os
hf_dataset_identifier = "sayakpaul/ucf101-subset"
filename = "UCF101_subset.tar.gz"
file_path = hf_hub_download(repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset", local_dir=".")
file_path

'UCF101_subset.tar.gz'

In [1]:
import tarfile
import os
with tarfile.open("UCF101_subset.tar.gz") as t:
     t.extractall("./data")

In [5]:
!wget https://github.com/olonok69/LLM_Notebooks/raw/main/video/fine_tune_ViViT/data_handling.py
!wget https://github.com/olonok69/LLM_Notebooks/raw/main/video/fine_tune_ViViT/model_configuration.py
!wget https://github.com/olonok69/LLM_Notebooks/raw/main/video/fine_tune_ViViT/preprocessing.py

--2024-11-09 13:51:32--  https://github.com/olonok69/LLM_Notebooks/raw/main/video/fine_tune_ViViT/data_handling.py
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/olonok69/LLM_Notebooks/main/video/fine_tune_ViViT/data_handling.py [following]
--2024-11-09 13:51:32--  https://raw.githubusercontent.com/olonok69/LLM_Notebooks/main/video/fine_tune_ViViT/data_handling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2915 (2.8K) [text/plain]
Saving to: ‘data_handling.py.1’

data_handling.py.1  100%[===================>]   2.85K  --.-KB/s    in 0s      

2024-11-09 13:51:33 (22.7 MB/s)

In [ ]:
%pip install transformers datasets==2.21.0 huggingface_hub

In [2]:
from transformers import TrainingArguments
from transformers import Trainer, TrainingArguments, AdamW
from model_configuration import *
from transformers import Trainer
from preprocessing import create_dataset
from data_handling import frames_convert_and_create_dataset_dictionary
from model_configuration import initialise_model
import wandb

/home/darth/.pyenv/versions/3.10.12/envs/major/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
from dotenv import load_dotenv
import os
env_path =  ".env"
load_dotenv(env_path)

True

## Base Model

In [ ]:
import model_configuration
from model_configuration import compute_metrics
import cv2
import av
from data_handling import sample_frame_indices, read_video_pyav

In [5]:
container = av.open("./data/UCF101_subset/test/ApplyEyeMakeup/v_ApplyEyeMakeup_g03_c01.avi")
container.streams.video[0].frames

209

In [6]:
%pip install moviepy -q


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import moviepy.editor
container = av.open("./data/UCF101_subset/test/ApplyEyeMakeup/v_ApplyEyeMakeup_g03_c01.avi")
indices = sample_frame_indices(clip_len=50, frame_sample_rate=2,seg_len=container.streams.video[0].frames)
video = read_video_pyav(container=container, indices=indices)

In [8]:
indices

array([ 74,  76,  78,  80,  82,  84,  86,  88,  90,  92,  94,  96,  98,
       100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 125,
       127, 129, 131, 133, 135, 137, 139, 141, 143, 145, 147, 149, 151,
       153, 155, 157, 159, 161, 163, 165, 167, 169, 171, 173])

In [9]:
video.shape

(50, 224, 224, 3)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [22]:
path_files = "data/UCF101_subset"
video_dict, class_labels = frames_convert_and_create_dataset_dictionary(path_files)


Processing file data/UCF101_subset/train/BabyCrawling/v_BabyCrawling_g08_c04.avi number of Frames: 95
Processing file data/UCF101_subset/train/BabyCrawling/v_BabyCrawling_g20_c05.avi number of Frames: 178
Processing file data/UCF101_subset/train/BabyCrawling/v_BabyCrawling_g23_c01.avi number of Frames: 116
Processing file data/UCF101_subset/train/BabyCrawling/v_BabyCrawling_g20_c07.avi number of Frames: 250
Processing file data/UCF101_subset/train/BabyCrawling/v_BabyCrawling_g17_c01.avi number of Frames: 141
Processing file data/UCF101_subset/train/BabyCrawling/v_BabyCrawling_g06_c04.avi number of Frames: 113
Processing file data/UCF101_subset/train/BabyCrawling/v_BabyCrawling_g02_c03.avi number of Frames: 117
Processing file data/UCF101_subset/train/BabyCrawling/v_BabyCrawling_g25_c01.avi number of Frames: 293
Processing file data/UCF101_subset/train/BabyCrawling/v_BabyCrawling_g20_c03.avi number of Frames: 263
Processing file data/UCF101_subset/train/BabyCrawling/v_BabyCrawling_g15_c

In [23]:
len(video_dict)

195

In [24]:
video_dict[0].keys()

dict_keys(['video', 'labels'])

In [25]:
video_dict[0]['video'].shape

(10, 224, 224, 3)

In [26]:
video_dict[0]['labels']

'BabyCrawling'

In [27]:
num_frames, height, width, channels =  video_dict[0]['video'].shape
num_frames, height, width, channels 

(10, 224, 224, 3)

## Display Video Sample

In [28]:
class_labels = sorted(class_labels)
label2id = {label: i for i, label in enumerate(class_labels)}
id2label = {i: label for label, i in label2id.items()}

print(f"Unique classes: {list(label2id.keys())}.")

Unique classes: ['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam'].


In [29]:
shuffled_dataset = create_dataset(video_dict)

Map: 100%|██████████| 195/195 [01:31<00:00,  2.12 examples/s]


In [30]:
shuffled_dataset['train'].features


{'labels': ClassLabel(names=['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam'], id=None),
 'pixel_values': Sequence(feature=Sequence(feature=Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None)}

In [31]:
model = model_configuration.initialise_model(shuffled_dataset, device)


Some weights of VivitForVideoClassification were not initialized from the model checkpoint at google/vivit-b-16x2-kinetics400 and are newly initialized because the shapes did not match:
- vivit.embeddings.position_embeddings: found shape torch.Size([1, 3137, 768]) in the checkpoint and torch.Size([1, 981, 768]) in the model instantiated
- classifier.weight: found shape torch.Size([400, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([400]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
%pip install accelerate==1.1.1

In [32]:
training_output_dir = "/tmp/results"
training_args = TrainingArguments(
    output_dir=training_output_dir,         
    num_train_epochs=3,             
    per_device_train_batch_size=2,   
    per_device_eval_batch_size=2,    
    learning_rate=5e-05,            
    weight_decay=0.01,              
    logging_dir="./logs",           
    logging_steps=10,                
    seed=42,                       
    eval_strategy="steps",    
    eval_steps=10,                   
    warmup_steps=int(0.1 * 20),      
    optim="adamw_torch",          
    lr_scheduler_type="linear",      
    fp16=True,  
    report_to="wandb"
)

In [33]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-05, betas=(0.9, 0.999), eps=1e-08)
# Define the trainer
trainer = Trainer(
    model=model,                      
    args=training_args,              
    train_dataset=shuffled_dataset["train"],      
    eval_dataset=shuffled_dataset["test"],       
    optimizers=(optimizer, None),  
    compute_metrics = compute_metrics
)

In [36]:
wandb_key =  os.getenv("WANDB_API_KEY")
wandb.login(key=wandb_key)

PROJECT = "ViViT"
MODEL_NAME = "google/vivit-b-16x2-kinetics400"
DATASET = "sayakpaul/ucf101-subset"

wandb.init(project=PROJECT, # the project I am working on
           tags=[MODEL_NAME, DATASET],
           notes ="Fine tuning ViViT with ucf101-subset")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [37]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-05, betas=(0.9, 0.999), eps=1e-08)
# Define the trainer
trainer = Trainer(
    model=model,                      
    args=training_args,              
    train_dataset=shuffled_dataset["train"],      
    eval_dataset=shuffled_dataset["test"],       
    optimizers=(optimizer, None),  
    compute_metrics = compute_metrics
)

In [38]:
with wandb.init(project=PROJECT, job_type="train", # the project I am working on
           tags=[MODEL_NAME, DATASET],
           notes =f"Fine tuning {MODEL_NAME} with {DATASET}."):
           train_results = trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
  4%|▍         | 10/264 [00:11<04:20,  1.03s/it]

{'loss': 1.648, 'grad_norm': 33.411678314208984, 'learning_rate': 4.885496183206107e-05, 'epoch': 0.11}


                                                
  4%|▍         | 10/264 [00:20<04:20,  1.03s/it]

{'eval_loss': 1.286291480064392, 'eval_accuracy': 0.45, 'eval_runtime': 9.2477, 'eval_samples_per_second': 2.163, 'eval_steps_per_second': 1.081, 'epoch': 0.11}


  8%|▊         | 20/264 [00:30<04:39,  1.15s/it]

...

                                                 
 98%|█████████▊| 260/264 [08:29<00:04,  1.14s/it]

{'eval_loss': 0.018836164847016335, 'eval_accuracy': 1.0, 'eval_runtime': 9.2971, 'eval_samples_per_second': 2.151, 'eval_steps_per_second': 1.076, 'epoch': 2.95}


100%|██████████| 264/264 [08:32<00:00,  1.94s/it]


{'train_runtime': 512.519, 'train_samples_per_second': 1.024, 'train_steps_per_second': 0.515, 'train_loss': 0.29176113641623297, 'epoch': 3.0}


eval/accuracy,▁▄▅▅▅▇▆▄▇▇▅▇▇██████▇██████
eval/loss,█▇▅▄▄▃▃▄▂▂▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▅▁▄▃██▅▇▃▆▆▃▇▄▆▆▃▆▅▅▅▅▆▃▆▆
eval/samples_per_second,▄█▅▆▁▁▄▂▆▂▃▆▂▅▃▃▅▃▄▄▄▄▃▆▃▃
eval/steps_per_second,▄█▅▆▁▁▄▂▆▃▃▆▁▅▃▃▆▃▄▄▅▄▃▆▃▃
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/grad_norm,▆█▄▅▅▃▅▆▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,██▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▁▁
train/loss,█▆▆▄▃▃▃▃▃▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁
eval/accuracy,1


In [39]:
trainer.save_model("model")
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

***** train metrics *****
  epoch                    =         3.0
  total_flos               = 384166470GF
  train_loss               =      0.2918
  train_runtime            =  0:08:32.51
  train_samples_per_second =       1.024
  train_steps_per_second   =       0.515


In [40]:
custom_path = "./model"


In [41]:
with wandb.init(project=PROJECT, job_type="models"):
  artifact = wandb.Artifact("ViViT-Fine-tuned", type="model")
  artifact.add_dir(custom_path)
  wandb.save(custom_path)
  wandb.log_artifact(artifact)

wandb: Adding directory to artifact (./model)... Done. 0.5s
